In [5]:
import pandas as pd

df = pd.read_excel('C:\\Users\\sktsa\\Projects\\coursera-advance-tensorflow-techniques\\data\\ENB2012_data.xlsx')
df = df.drop(columns=['Unnamed: 10', 'Unnamed: 11'])
df = df.sample(frac=1).reset_index(drop=True)
df

KeyError: "['Unnamed: 10' 'Unnamed: 11'] not found in axis"

In [ ]:
import numpy as np


def format_output(data):
    y1 = data.pop('Y1')
    y1 = np.array(y1)
    y2 = data.pop('Y2')
    y2 = np.array(y2)
    return y1, y2

In [ ]:
def norm(x):
    return (x - train_stats['mean']) / train_stats['std']

In [ ]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(df, test_size=0.2)
train_stats = train.describe()


In [ ]:
train_stats.pop('Y1')
train_stats.pop('Y2')

train_stats = train_stats.transpose()
train_stats

,count,mean,std,min,25%,50%,75%,max
X1,614.0,0.762671,0.106267,0.62,0.66,0.74,0.82,0.98
X2,614.0,673.151466,88.602384,514.50,612.50,686.00,759.50,808.50
X3,614.0,318.899023,43.740126,245.00,294.00,318.50,343.00,416.50
X4,614.0,177.126221,45.326383,110.25,122.50,220.50,220.50,220.50
X5,614.0,5.221498,1.751195,3.50,3.50,3.50,7.00,7.00
X6,614.0,3.488599,1.114505,2.00,2.00,3.00,4.00,5.00
X7,614.0,0.233958,0.133435,0.00,0.10,0.25,0.40,0.40
X8,614.0,2.770358,1.555195,0.00,1.00,3.00,4.00,5.00


In [ ]:
train_y = format_output(train)
test_y = format_output(test)

In [ ]:
norm_train_x = norm(train)
norm_test_x = norm(test)

In [ ]:
norm_train_x

,X1,X2,X3,X4,X5,X6,X7,X8
633,-0.966160,0.974562,-0.009123,0.956921,-0.983042,0.458859,0.120225,-1.138352
640,-0.025135,-0.131503,2.231383,-1.205175,1.015593,-1.335660,-1.003913,-0.495345
617,0.257173,-0.408019,0.551004,-0.664651,1.015593,-0.438400,0.120225,-1.138352
72,-1.342570,1.527595,1.111130,0.956921,-0.983042,-0.438400,1.244364,1.433674
649,2.045121,-1.790600,-0.569249,-1.475437,1.015593,-0.438400,0.120225,-0.495345
...,...,...,...,...,...,...,...,...
282,-0.025135,-0.131503,2.231383,-1.205175,1.015593,1.356119,0.120225,0.790667
152,-0.025135,-0.131503,2.231383,-1.205175,1.015593,1.356119,0.120225,-0.495345
84,0.915890,-0.961052,-0.569249,-0.664651,1.015593,-0.438400,0.120225,1.433674
22,0.915890,-0.961052,-0.569249,-0.664651,1.015593,0.458859,0.120225,0.790667


In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Input, Flatten
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.metrics import RootMeanSquaredError

In [ ]:
input_layer = Input(shape=(len(train.columns),))
x = Dense(128, activation='relu')(input_layer)
x = Dense(128, activation='relu')(x)

y1_output = Dense(1, name='y1_output')(x)
x = Dense(64, activation='relu')(x)
y2_output = Dense(1, name='y2_output')(x)

model = Model(inputs=input_layer, outputs=[y1_output, y2_output])


In [ ]:
model.summary()

Model: "model_3"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_4 (InputLayer)           [(None, 8)]          0           []                               
                                                                                                  
 dense_9 (Dense)                (None, 128)          1152        ['input_4[0][0]']                
                                                                                                  
 dense_10 (Dense)               (None, 128)          16512       ['dense_9[0][0]']                
                                                                                                  
 dense_11 (Dense)               (None, 64)           8256        ['dense_10[0][0]']               
                                                                                            

In [ ]:
optimizer = SGD(lr=0.001)
model.compile(optimizer=optimizer,
              loss={
                  'y1_output': 'mse',
                  'y2_output': 'mse'
              },
              metrics={
                  'y1_output': RootMeanSquaredError(),
                  'y2_output': RootMeanSquaredError()
              })


C:\Users\sktsa\tflow\lib\site-packages\keras\optimizer_v2\gradient_descent.py:102: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


In [ ]:
r = model.fit(norm_train_x, train_y, epochs=1000, batch_size=10, validation_data=(norm_test_x, test_y))

Epoch 1/1000
62/62 [==============================] - 2s 9ms/step - loss: 245.1686 - y1_output_loss: 115.3489 - y2_output_loss: 129.8197 - y1_output_root_mean_squared_error: 10.7401 - y2_output_root_mean_squared_error: 11.3938 - val_loss: 40.4416 - val_y1_output_loss: 16.4753 - val_y2_output_loss: 23.9662 - val_y1_output_root_mean_squared_error: 4.0590 - val_y2_output_root_mean_squared_error: 4.8955
Epoch 2/1000
62/62 [==============================] - 0s 6ms/step - loss: 40.4054 - y1_output_loss: 13.9023 - y2_output_loss: 26.5031 - y1_output_root_mean_squared_error: 3.7286 - y2_output_root_mean_squared_error: 5.1481 - val_loss: 61.0879 - val_y1_output_loss: 17.8408 - val_y2_output_loss: 43.2471 - val_y1_output_root_mean_squared_error: 4.2238 - val_y2_output_root_mean_squared_error: 6.5763
Epoch 3/1000
62/62 [==============================] - 0s 6ms/step - loss: 29.1138 - y1_output_loss: 11.0343 - y2_output_loss: 18.0795 - y1_output_root_mean_squared_error: 3.3218 - y2_output_root_mean

In [ ]:
loss, y1_loss, y2_loss, y1_rmse, y2_rmse = model.evaluate(x=norm_test_x, y=test_y)

5/5 [==============================] - 0s 4ms/step - loss: 0.8758 - y1_output_loss: 0.1942 - y2_output_loss: 0.6817 - y1_output_root_mean_squared_error: 0.4406 - y2_output_root_mean_squared_error: 0.8256


In [ ]:
loss, y1_loss, y2_loss, y1_rmse, y2_rmse

(0.8758342862129211,
 0.1941511631011963,
 0.6816831231117249,
 0.4406258761882782,
 0.8256410360336304)

In [ ]:
import seaborn as sns
df = pd.DataFrame.from_dict(r.history)